In [17]:
!pip install beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
!pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
from bs4 import BeautifulSoup
import requests
import re
from datetime import datetime
import pandas as pd

In [20]:
#This first URL is to obtain the total of pages we will iterate through
#'https://www.trustpilot.com/review/www.axa.co.uk and https://www.trustpilot.com/review/www.axa.co.uk?page=1 work exactly the same '
url= 'https://www.trustpilot.com/review/www.axa.co.uk?page=1'
results=requests.get(url)
doc=BeautifulSoup(results.text, 'html.parser')

#Obtains the total of pages
pages=doc.find('a', {'name':'pagination-button-last'})
total_pages=int(pages.string)



## Obtaining reviews paragraphs

In [21]:
#Only the paragraphs belonging to the correct class reviews
#paragraphs2=doc.find_all('p',{'class':"typography_typography__QgicV typography_body__9UBeQ typography_color-black__5LYEn typography_weight-regular__TWEnf typography_fontstyle-normal__kHyN3"})


In [22]:
#text for obtaining the text even if it has breaks <br>
#if I use .string instead of .text returns None when <br> present
# for para in paragraphs2:
#   print(para.text)

## Obtaining the date of the reviews

In [23]:
# dates=doc.find_all('time')
# len(dates)

## Obtaining data by descendants

In [24]:
# section=doc.find('section',{'class':"styles_reviewsContainer__3_GQw"})

## Should iterate through these articles
# articles=doc.find_all('article',{'class':"paper_paper__1PY90 paper_square__lJX8a card_card__lQWDv styles_reviewCard__hcAvl"})
# articles=doc.find_all('article',{'class':"paper_paper__1PY90 paper_square__lJX8a card_card__lQWDv styles_reviewCard__hcAvl styles_raised__HxiOi"})

#wrapers are the base, they are the containers of the reviews



In [25]:
## Obtaining date
# times=wrapers[0].contents[0].contents[1].find('time')
# t=times['datetime']
# fecha=re.search("([0-9]{4}\-[0-9]{2}\-[0-9]{2})", t)
# fecha[0]
# print (datetime.fromisoformat(fecha[0]).date())

In [26]:
# for w in wrapers[3].contents[1]:
#   print(w)


In [27]:
# wrapers[3].contents[1].find('p').text

In [28]:
# for page in range(total_pages):
lista=[]
for page in range(1,total_pages+1):
  #print(f'this is page{page}')


  url= f'https://www.trustpilot.com/review/www.axa.co.uk?page={page}'
  results=requests.get(url)
  doc=BeautifulSoup(results.text, 'html.parser')

#wrapers are the base of everything there is on content wrapper of class "styles_reviewContentwrapper__zH_9M
#for every review and they nest rating, time, title and the review text
  wrapers=doc.find_all('section',{'class':"styles_reviewContentwrapper__zH_9M"})
  
  
  for i in range(len(wrapers)):
    #this gets the rating wrapers[i].contents[0]['data-service-review-rating']
    rating=int(wrapers[i].contents[0]['data-service-review-rating'])
    #this gets the date
    times_tag=wrapers[i].contents[0].contents[1].find('time')
    time_string_long=times_tag['datetime']
    time_string_regex=re.search("([0-9]{4}\-[0-9]{2}\-[0-9]{2})", time_string_long)
    time_string_short=time_string_regex[0]
    date= datetime.fromisoformat(time_string_short).date()
    #Review title
    title=wrapers[i].contents[1].find('h2').text
    #Review tex
    if wrapers[i].contents[1].find('p'):
      review=wrapers[i].contents[1].find('p').text
    else:
      review= None


    data=[rating,date,title,review]

    lista.append(data) 
  
# lista

In [29]:
df=pd.DataFrame(lista, columns=[['rating','date','title','review']])
df

,rating,date,title,review
0,5,2022-07-30,The Lady that helped us,The Lady that helped us change our policy deta...
1,5,2022-07-30,All done on my phone,All done on my phone. Nice and easy.
2,5,2022-07-30,It was easy & excellent service,It was easy & excellent service. Informative ...
3,4,2022-07-30,I would have given 5 stars as the…,I would have given 5 stars as the person was v...
4,5,2022-07-30,simple application form for home…,simple application form for home insurance onl...
...,...,...,...,...
3335,5,2021-12-23,Had to change some details on policy,"Had to change some details on policy, staff we..."
3336,5,2021-12-23,AXA have and remain a pleasure to deal…,AXA have and remain a pleasure to deal with
3337,4,2021-12-23,Great price great cover just a shame…,Great price great cover just a shame they don'...
3338,5,2021-12-23,Extremely easy company,"Extremely easy company, app to use. Updating a..."


In [30]:
# df.groupby('date')